In [1]:
import os
import threading
import time

import ipywidgets
from IPython.display import display

from utils.multi_model_inference import DecodeMultiInferResult
from utils.utils import bgr8_to_jpeg, img_path_to_msg

In [2]:
width = 224
height = 224

result_decode = DecodeMultiInferResult("./0420school_A/val_multi-model_inference.json")
circled_img, json_dict = result_decode.get_img_value()
img_name, project, category, dataset = img_path_to_msg(json_dict["img_path"])
model_output = list(json_dict["model_output"].values())

4 models in ./0420school_A/val_multi-model_inference.json
successful load 298 images


In [4]:
image_widget = ipywidgets.Image(value=bgr8_to_jpeg(circled_img), format="jpg", width=width, height=height)
    
Source_CH1_widget = ipywidgets.BoundedFloatText(description='CH1', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
Source_CH2_widget = ipywidgets.BoundedFloatText(description='CH2', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})

count_widget = ipywidgets.IntText(description='count', disabled=True, value=len(result_decode), style={'description_width': 'initial'})


# 定义播放按钮
play_button = ipywidgets.ToggleButton(description='play', value=False, disabled=False)
change_bool = play_button.value
def play():
    global change_bool
    while True:
        if change_bool:

            circled_img, json_dict = result_decode.get_img_next(step_widget.value)
            image_widget.value = bgr8_to_jpeg(circled_img)
            Source_CH1_widget.value = json_dict["source_value"]["1"]
            Source_CH2_widget.value = json_dict["source_value"]["2"]
            index_widget.value = result_decode.get_index()
            filename_list_widget.value = os.path.basename(json_dict["img_path"])
            model_output = list(json_dict["model_output"].values())
            model_1_CH1_widget.value = model_output[0]["detected_value"]["1"]
            model_1_CH2_widget.value = model_output[0]["detected_value"]["2"]
            model_2_CH1_widget.value = model_output[1]["detected_value"]["1"]
            model_2_CH2_widget.value = model_output[1]["detected_value"]["2"]
            model_3_CH1_widget.value = model_output[2]["detected_value"]["1"]
            model_3_CH2_widget.value = model_output[2]["detected_value"]["2"]
            model_4_CH1_widget.value = model_output[3]["detected_value"]["1"]
            model_4_CH2_widget.value = model_output[3]["detected_value"]["2"]

        time.sleep(0.15)
execute_thread = threading.Thread(target=play)
execute_thread.start()

def playClick(change):
    global change_bool
    change_bool = change.new
    if play_button.description == "play":
        play_button.description = "stop"
    else:
        play_button.description = "play"
play_button.observe(playClick, names='value')

forward_button = ipywidgets.Button(description='next')
def forward(c):
    circled_img, json_dict = result_decode.get_img_next(step_widget.value)
    set_image_widget(circled_img, json_dict)
    index_widget.value = result_decode.get_index()
    filename_list_widget.value = os.path.basename(json_dict["img_path"])
forward_button.on_click(forward)

back_button = ipywidgets.Button(description='back')
def back(c):
    circled_img, json_dict = result_decode.get_img_next(-step_widget.value)
    set_image_widget(circled_img, json_dict)
    index_widget.value = result_decode.get_index()
    filename_list_widget.value = os.path.basename(json_dict["img_path"])
back_button.on_click(back)

step_widget = ipywidgets.BoundedIntText(value=1, min=0, max=count_widget.value, step=1)

filename_list_widget = ipywidgets.Dropdown(options=result_decode.get_img_name_list(), value=result_decode.get_img_name_list()[0] , description='img path', style={'description_width': 'initial'})
def filename_list_update(change):
    new_filename = change.new
    result_decode.index = result_decode.get_img_name_list().index(new_filename)
    circled_img, json_dict = result_decode.get_img_value()
    set_image_widget(circled_img, json_dict)
    index_widget.value = result_decode.get_index()
filename_list_widget.observe(filename_list_update, names='value')

index_widget = ipywidgets.IntText(description='index', value=0,style={'description_width': 'initial'})
def index_change(change):
    new_index = change.new
    if new_index > len(result_decode):
        new_index -= len(result_decode)
    result_decode.index = new_index
    circled_img, json_dict = result_decode.get_img_value()
    set_image_widget(circled_img, json_dict)
    filename_list_widget.value = os.path.basename(json_dict["img_path"])
index_widget.observe(index_change, names='value')

def set_image_widget(circled_img, json_dict):
    image_widget.value = bgr8_to_jpeg(circled_img)
    Source_CH1_widget.value = json_dict["source_value"]["1"]
    Source_CH2_widget.value = json_dict["source_value"]["2"]
    model_output = list(json_dict["model_output"].values())
    model_1_CH1_widget.value = model_output[0]["detected_value"]["1"]
    model_1_CH2_widget.value = model_output[0]["detected_value"]["2"]
    model_2_CH1_widget.value = model_output[1]["detected_value"]["1"]
    model_2_CH2_widget.value = model_output[1]["detected_value"]["2"]
    model_3_CH1_widget.value = model_output[2]["detected_value"]["1"]
    model_3_CH2_widget.value = model_output[2]["detected_value"]["2"]
    model_4_CH1_widget.value = model_output[3]["detected_value"]["1"]
    model_4_CH2_widget.value = model_output[3]["detected_value"]["2"]

In [5]:
RGB, HEX = result_decode.get_model_color(model_name=model_output[0]["model_path"])
model_1_color_choice = ipywidgets.ColorPicker(concise=True, value=HEX, style={'description_width': 'initial'})
model_1_model_name_widget = ipywidgets.Text(value=model_output[0]["model_path"], disabled=True, style={'description_width': 'initial'})
model_1_CH1_widget = ipywidgets.BoundedFloatText(description='CH1', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
model_1_CH2_widget = ipywidgets.BoundedFloatText(description='CH2', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})

model_1_color_choice.layout.width = '30px'
model_1_model_name_widget.layout.width = "270px"
model_1_CH1_widget.layout.width = "150px"
model_1_CH2_widget.layout = model_1_CH1_widget.layout

model_1_box = ipywidgets.VBox([ipywidgets.HBox([model_1_color_choice, model_1_model_name_widget]),
                                      ipywidgets.HBox([model_1_CH1_widget,model_1_CH2_widget])])
display(model_1_box)

In [6]:
RGB, HEX = result_decode.get_model_color(model_name=model_output[1]["model_path"])
model_2_color_choice = ipywidgets.ColorPicker(concise=True, value=HEX, style={'description_width': 'initial'})
model_2_model_name_widget = ipywidgets.Text(value=model_output[1]["model_path"], disabled=True, style={'description_width': 'initial'})
model_2_CH1_widget = ipywidgets.BoundedFloatText(description='CH1', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
model_2_CH2_widget = ipywidgets.BoundedFloatText(description='CH2', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})

model_2_color_choice.layout.width = '30px'
model_2_model_name_widget.layout.width = "270px"
model_2_CH1_widget.layout.width = "150px"
model_2_CH2_widget.layout = model_2_CH1_widget.layout

model_2_box = ipywidgets.VBox([ipywidgets.HBox([model_2_color_choice, model_2_model_name_widget]),
                                      ipywidgets.HBox([model_2_CH1_widget,model_2_CH2_widget])])
display(model_2_box)

In [7]:
RGB, HEX = result_decode.get_model_color(model_name=model_output[2]["model_path"])
model_3_color_choice = ipywidgets.ColorPicker(concise=True, value=HEX, style={'description_width': 'initial'})
model_3_model_name_widget = ipywidgets.Text(value=model_output[2]["model_path"], disabled=True, style={'description_width': 'initial'})
model_3_CH1_widget = ipywidgets.BoundedFloatText(description='CH1', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
model_3_CH2_widget = ipywidgets.BoundedFloatText(description='CH2', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})

model_3_color_choice.layout.width = '30px'
model_3_model_name_widget.layout.width = "270px"
model_3_CH1_widget.layout.width = "150px"
model_3_CH2_widget.layout = model_3_CH1_widget.layout

model_3_box = ipywidgets.VBox([ipywidgets.HBox([model_3_color_choice, model_3_model_name_widget]),
                                      ipywidgets.HBox([model_3_CH1_widget,model_3_CH2_widget])])
display(model_3_box)

In [8]:
RGB, HEX = result_decode.get_model_color(model_name=model_output[3]["model_path"])
model_4_color_choice = ipywidgets.ColorPicker(concise=True, value=HEX, style={'description_width': 'initial'})
model_4_model_name_widget = ipywidgets.Text(value=model_output[3]["model_path"], disabled=True, style={'description_width': 'initial'})
model_4_CH1_widget = ipywidgets.BoundedFloatText(description='CH1', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
model_4_CH2_widget = ipywidgets.BoundedFloatText(description='CH2', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})

model_4_color_choice.layout.width = '30px'
model_4_model_name_widget.layout.width = "270px"
model_4_CH1_widget.layout.width = "150px"
model_4_CH2_widget.layout = model_4_CH1_widget.layout

model_4_box = ipywidgets.VBox([ipywidgets.HBox([model_4_color_choice, model_4_model_name_widget]),
                                      ipywidgets.HBox([model_4_CH1_widget,model_4_CH2_widget])])
display(model_4_box)

In [9]:
play_button.layout.width = '{}px'.format((width/4) - 3)
forward_button.layout = play_button.layout
step_widget.layout = play_button.layout
back_button.layout = play_button.layout

filename_list_widget.layout.width = "{}px".format(width)
index_widget.layout = filename_list_widget.layout
count_widget.layout = filename_list_widget.layout

Source_CH1_widget.layout.width = '{}px'.format((width/2) - 3)
Source_CH2_widget.layout = Source_CH1_widget.layout

In [10]:
RGB, HEX = result_decode.get_model_color(model_name="source_value")
source_color_choice = ipywidgets.ColorPicker(concise=True, value=HEX, description='source color', 
                                             style={'description_width': 'initial'})
source_color_choice.layout.width = '30px'
img_widget = ipywidgets.VBox([
    image_widget,
    ipywidgets.HBox([play_button, back_button, step_widget, forward_button]),
    ipywidgets.HBox([Source_CH1_widget, Source_CH2_widget]),
    filename_list_widget,
    index_widget,
    count_widget,])

# noinspection PyTypeChecker
display(ipywidgets.HBox([img_widget, ipywidgets.VBox([source_color_choice, model_1_box, model_2_box, model_3_box, model_4_box])]))